### Barrier

In [ ]:
## 栅栏

In [2]:
import time 
import logging
import threading
import importlib

importlib.reload(logging)
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s  [%(threadName)s] %(message)s')

In [15]:
logging.info('hello')

2017-05-02 07:24:01,992 INFO  [MainThread] hello


In [ ]:
## 三个哥们约定去喝酒，A去骗钱，B去向领导请假，C单身，约定大家准备好后，酒吧门口见，只有三个都到了，才一起进入！

In [10]:
def worker(barrier:threading.Barrier):
    logging.info('waitting for {} threads'.format(barrier.n_waiting))  ## 这段是各自要干的事情
    try:                                                ## n_waiting就是现在有多少个线程在waiting
        worker_id = barrier.wait()  ## 这段就是在酒吧门口等候   等待一下
    except threading.BrokenBarrierError:
        logging.warning('aborting')
    else:      ## 这种语句就是 只有说没有抛出except，才执行这条语句 和下面的注释是等效的
        logging.info('after barrier {}'.format(worker_id))  ## 最后走完这段（剩下的流程）
        

# def worker(barrier:threading.Barrier):
#     logging.info('waitting for {} threads'.format(barrier.n_waiting)) 
#     try:
#         worker_id = barrier.wait()  
#         logging.info('after barrier {}'.format(worker_id))
#     except threading.BrokenBarrierError:
#         logging.warning('aborting')

In [11]:
barrier = threading.Barrier(3)  ## 这个参数是设置拦多少个的  这里是只要3个到齐就OK

In [16]:
for x in range(3):     ## 启动若干线程
    threading.Thread(target=worker, name='worker-{}'.format(x), args=(barrier,)).start()
    time.sleep(0.5)
logging.info('started')

2017-05-02 07:34:09,752 INFO  [worker-0] waitting for 0 threads
2017-05-02 07:34:10,254 INFO  [worker-1] waitting for 1 threads
2017-05-02 07:34:10,788 INFO  [worker-2] waitting for 2 threads
2017-05-02 07:34:10,791 INFO  [worker-2] after barrier 2
2017-05-02 07:34:10,791 INFO  [worker-1] after barrier 1
2017-05-02 07:34:10,793 INFO  [worker-0] after barrier 0
2017-05-02 07:34:11,291 INFO  [MainThread] started


栅栏，凑齐一波线程，才继续往下走

看barrier的几个属性

In [18]:
barrier.n_waiting   ## 现在有多少个线程在等待

0

In [ ]:
barrier.abort()  ## 这是个方法  通知已经在等待的线程不必再等了，我执行不到wait

In [19]:
## 看一下abort()这个的应用

barrier = threading.Barrier(3)

In [20]:
def abort(barrier):
    logging.info('aborting')
    barrier.abort()

In [21]:
for x in range(2):     ##  这里先启动两个线程   看效果，正在等待   应该是要小于3个的，要不然3个都到齐了，就不等了，继续执行了
    threading.Thread(target=worker, name='worker-{}'.format(x), args=(barrier,)).start()

2017-05-02 07:45:01,431 INFO  [worker-0] waitting for 0 threads
2017-05-02 07:45:01,433 INFO  [worker-1] waitting for 1 threads


In [22]:
barrier.abort()    ## 任意一个线程不再等待，中断它，中断后，都执行except 这段了  ## abort(中止)

2017-05-02 07:45:36,257 WARNING  [worker-1] aborting
2017-05-02 07:45:36,257 WARNING  [worker-0] aborting


当我们执行的wait的线程数等于我们给定的这个值的时候，它就开始**往下执行了**；如果没到这个数，而其中一个执行了abort，wait会触发BrokenBarrierError的异常

当abort方法被执行的时候，wait方法会抛出BrokenBarrieError 异常

栅栏用在什么场景呢？
* 10种工作，每个线程负责一种，只有10个线程都初始化完成，才能工作。
* 有10个工作，完成其中的6个，就算完成。(这个时候，每个工作用一个线程来做)

In [23]:
help(barrier.wait)    ## 所有wait方法，都可以设 timeout

Help on method wait in module threading:

wait(timeout=None) method of threading.Barrier instance
    Wait for the barrier.
    
    When the specified number of threads have started waiting, they are all
    simultaneously awoken. If an 'action' was provided for the barrier, one
    of the threads will have executed that callback prior to returning.
    Returns an individual index number from 0 to 'parties-1'.



In [24]:
barrier.wait(1)   ## 因为已经执行过abort方法了，所以抛出异常  abort后的任何wait都会抛出异常

BrokenBarrierError: 

In [25]:
barrier.reset()   ## reset后，试试

In [26]:
barrier.wait(3)   ## 当到时间的时候，也是会抛出异常

BrokenBarrierError: 

注：栅栏的timeout不像其它的，其它的都是返回False，栅栏是抛出**异常**！

栅栏里，不论是是timeout还是abort，都是抛出异常的！

栅栏用的最多的，就是并发初始化。其它时候用的比较少。

并发初始化用的是比较常用的，比如说要load一个2G的文件，就可以用10个线程，每个线程处理200M，都处理完成后，才进入主循环。(当然有前提条件，初始化工作可以乱序的时候才行)

In [ ]:
讲这么多，同步，就是线程之间的通讯，这些线程总是有某种关联才需要同步的，同步是会打破并发的。
所以，如果这些线程完全没有关联的话，完全没必有用到同步的。
这些线程要是独立的话，是不需要用到同步的。

同步通常意味着阻塞。

### Semaphore    信号量

信号量跟锁非常像

In [27]:
s = threading.Semaphore(3)   ## 定义一个信号量为3的信号量

In [28]:
s.acquire()

True

In [29]:
s.acquire(False)

True

In [30]:
s.acquire(False)

True

In [31]:
s.acquire(False)    ## 第4 次的时候，变成False

False

锁是信号量的特例：为1 的信号量

信号量在不同的线程里都是可以进入的。

锁是只有一个线程！！

In [ ]:
## 例：连接池的类

In [44]:
class Pool:
    def __init__(self, num):
        self.num = num
        self.conns = [self._make_connect(x) for x in range(num)]    ##  初始化的时候，先创建起来
        self.s = threading.Semaphore(num)   ## 创建信号量的对象   最多有num个连接
    
    def _make_connect(self, name):    ## 创建连接的方法
        return name
    
    def get(self):
        self.s.acquire()            ## 加锁
        return self.conns.pop()
    
    def return_resource(self, conn):
        self.conns.insert(0,conn) 
        self.s.release()
    

In [45]:
import random

def worker(pool):
    logging.info('started')
    name = pool.get()
    logging.info('got connect {}'.format(name))
    time.sleep(random.randint(1,3))
    pool.return_resource(name)
    logging.info('return resource {}'.format(name))

In [46]:
pool = Pool(3)   ## 池子大小为3

In [47]:
for x in range(5):   ## 启动5 个worker         ##  这就实现了一个资源的竞争
    threading.Thread(target=worker, args=(pool,), name='worker-{}'.format(x)).start()

2017-05-02 18:50:47,730 INFO  [worker-0] started
2017-05-02 18:50:47,734 INFO  [worker-1] started
2017-05-02 18:50:47,751 INFO  [worker-1] got connect 1
2017-05-02 18:50:47,738 INFO  [worker-3] started
2017-05-02 18:50:47,759 INFO  [worker-3] got connect 0
2017-05-02 18:50:47,740 INFO  [worker-0] got connect 2
2017-05-02 18:50:47,734 INFO  [worker-2] started
2017-05-02 18:50:47,738 INFO  [worker-4] started
2017-05-02 18:50:48,763 INFO  [worker-3] return resource 0
2017-05-02 18:50:48,764 INFO  [worker-2] got connect 0
2017-05-02 18:50:48,769 INFO  [worker-0] return resource 2
2017-05-02 18:50:48,770 INFO  [worker-4] got connect 2
2017-05-02 18:50:50,761 INFO  [worker-1] return resource 1
2017-05-02 18:50:50,776 INFO  [worker-4] return resource 2
2017-05-02 18:50:51,786 INFO  [worker-2] return resource 0


信号量也是对资源的保护，但是和锁不一样的地方在于，锁限制只有一个线程可以访问共享资源，而信号量限制指定个线程可以访问共享资源。

锁是特化的信号量

信号量通常用来做连接池类的东西